In [137]:
!pip install pypdf


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [138]:
from PyPDF2 import PdfReader

In [355]:
def extraer_texto_pagina(nombre_del_archivo,pagina):
    try:
        # Abrir el archivo PDF
        with open(nombre_del_archivo, "rb") as archivo_pdf:
            # Crear un objeto PdfReader
            pdf_reader = PdfReader(archivo_pdf)
            
            # Imprimir la cantidad de páginas del PDF
            print('*'*72)
            print(f"PAGINA {pagina}/{len(pdf_reader.pages)} del PDF {nombre_del_archivo}")
            print("-" * 52)


            # Devolver el objeto PdfReader
            return pdf_reader.pages[pagina-1].extract_text()
            
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_del_archivo}'")
        return None
    except Exception as e:
        print(f"Error al procesar el archivo '{nombre_del_archivo}': {e}")
        return None


In [155]:
nombre_archivo = r".\Diccionariospdf\YACHAKUQKUNAPA_SHIMI_QULLQA.pdf"

In [391]:
from PyPDF2 import PdfReader
import pandas as pd


In [204]:
import spacy

# Cargar el modelo en español de spacy
nlp = spacy.load('es_core_news_sm')

def es_palabra_espanol(palabra):
    # Procesar la palabra con el modelo de spacy
    doc = nlp(palabra)
    
    # Verificar si el idioma detectado por spacy es español ('es')
    return doc.lang_ == 'es'


In [289]:
def formar_palabra(palabras):
    pal=''
    for a in palabras: 
        if pal=='':
            pal=a.lower()
            continue
        pal=pal+' '+a.lower()  
    return pal

In [339]:
def obtener_sinonimos(linea):
    sinonimos = []
    partes = linea.split(',')
    index = len(partes) - 1
    for parte in partes[:index]:
        sinonimos.append(parte.strip().lower())
    palaux = partes[index].split(' ')
    pal = ''
    palabraquechua = ''
    for paux in palaux:
        if paux and paux[0].isupper():
            palabraquechua = paux
        if paux and not paux[0].isupper():
            if pal == '':
                pal = paux
            else:
                pal = pal + ' ' + paux
    sinonimos.append(pal)
    return sinonimos, palabraquechua.lower()

In [386]:
def retornar_diccionario(nombre_archivo, numero_pagina):
    texto=extraer_texto_pagina(nombre_archivo,numero_pagina)
    lineas = texto.splitlines()
    lineas_no_validas=0
    palabra_es=[]
    palabra_que=[]
    linea_auxiliar=''
    sinonimosaux=[]
    for linea in lineas:
        if linea.strip().isdigit() or (len(linea.strip()) == 1 and linea.strip().isalnum()) or ' - ' in linea:
            lineas_no_validas+=1
            continue
        
        palabras=linea.split()
        if ',' in linea :
                
                sinonimos, palabraquechua = obtener_sinonimos(linea)

                if not palabraquechua:
                    linea_auxiliar=linea
                    continue

                if linea_auxiliar:
                    linea_completa= linea_auxiliar+''+linea
                    print(linea_completa)
                    sinonimos, palabraquechua = obtener_sinonimos(linea_completa)
                    linea_auxiliar=''
                     
                for s in sinonimos:
                    palabra_es.append(s)
                    palabra_que.append(palabraquechua.lower())
                continue 

        if len(palabras)==2:
            palabra_es.append(palabras[0].lower())
            palabra_que.append(palabras[1].lower())
        

        
                # continue 
        if len(palabras)>2 and not ',' in linea :
                index_quechua=-1
                for idx, pix in enumerate(palabras[1:], start=1):
                    if pix[0].isupper():
                        index_quechua = idx
                        break
                if index_quechua<0:
                    linea_auxiliar=linea
 
                else: 
                    if linea_auxiliar:
                        palabra_es.append(linea_auxiliar+' '+formar_palabra(palabras[:index_quechua]))
                        palabra_que.append(formar_palabra(palabras[index_quechua:])) 
                        linea_auxiliar=''
                    else:
                        palabra_es.append(formar_palabra(palabras[:index_quechua]))
                        palabra_que.append(formar_palabra(palabras[index_quechua:])) 
                    
                    
                    
                
    print(f"cantidad de lineas: {len(lineas)-lineas_no_validas}")             
    print(f"cantidad de palabras español: {len(palabra_es)}")
    print(f"cantidad de palabras quechua: {len(palabra_que)}")
    return palabra_es, palabra_que
    

In [388]:

numero_pagina = 102
palabra_es, palabra_que = retornar_diccionario(nombre_archivo, numero_pagina)

#Imprimir los resultados
print('.'*10)
for e, q in zip(palabra_es, palabra_que):
    print(f"{e} : {q}") 

************************************************************************
PAGINA 102/129 del PDF .\Diccionariospdf\YACHAKUQKUNAPA_SHIMI_QULLQA.pdf
----------------------------------------------------
cantidad de lineas: 38
cantidad de palabras español: 46
cantidad de palabras quechua: 46
..........
abeja : wanquyru
ablandar : llaptsiy
abortar : shulluy
abrazar : waquy
abrir : kichay
abrir la boca : aayay
abuela : awila
abuelo : awilu
aceptar : aniy
estar de acuerdo : aniy
acusar : tumpay
culpar : tumpay
adelante : nawpa
antes : nawpa
tiempo antiguo : nawpa
adorar : mutsakuy
venerar : mutsakuy
adornar : allitsay
arreglar : allitsay
agonizante : wanunaykaq
agravarse : antsay
agregar : yapay
repetir : yapay
agua : yaku
aguar : yakuchay
aguila : anka
aguja de arriero : antakasha
ahogamiento : shinqashyay
ají : utsu
ala : ripra
alacrán : atuq kuru
alegrarse : kushikuy
alga : kushuru
algodón : utku
aliso : ramrash
aliso : wayaw
alma : hupay
espíritu : hupay
almohada : hawya
alpaca : paqu
alum

In [389]:
# Listas finales para almacenar todos los resultados
palabras_espanol = []
palabras_quechua = []

# Iterar desde la página 102 hasta la página 125
for numero_pagina in range(102, 127):
    palabra_es, palabra_que = retornar_diccionario(nombre_archivo, numero_pagina)
    
    # Agregar los resultados de la página actual a las listas finales
    palabras_espanol.extend(palabra_es)
    palabras_quechua.extend(palabra_que)



************************************************************************
PAGINA 102/129 del PDF .\Diccionariospdf\YACHAKUQKUNAPA_SHIMI_QULLQA.pdf
----------------------------------------------------
cantidad de lineas: 38
cantidad de palabras español: 46
cantidad de palabras quechua: 46
************************************************************************
PAGINA 103/129 del PDF .\Diccionariospdf\YACHAKUQKUNAPA_SHIMI_QULLQA.pdf
----------------------------------------------------
cantidad de lineas: 53
cantidad de palabras español: 63
cantidad de palabras quechua: 63
************************************************************************
PAGINA 104/129 del PDF .\Diccionariospdf\YACHAKUQKUNAPA_SHIMI_QULLQA.pdf
----------------------------------------------------
cantidad de lineas: 40
cantidad de palabras español: 49
cantidad de palabras quechua: 49
************************************************************************
PAGINA 105/129 del PDF .\Diccionariospdf\YACHAKUQKUNAPA_SHIMI_Q

In [390]:
# Imprimir los resultados finales
print("Resultados finales:")
print(f"cantidad de palabras español: {len(palabras_espanol)}")
print(f"cantidad de palabras quechua: {len(palabras_quechua)}")
print('.'*10)
for e, q in zip(palabras_espanol, palabras_quechua):
    print(f"{e} : {q}") 

Resultados finales:
cantidad de palabras español: 1143
cantidad de palabras quechua: 1143
..........
abeja : wanquyru
ablandar : llaptsiy
abortar : shulluy
abrazar : waquy
abrir : kichay
abrir la boca : aayay
abuela : awila
abuelo : awilu
aceptar : aniy
estar de acuerdo : aniy
acusar : tumpay
culpar : tumpay
adelante : nawpa
antes : nawpa
tiempo antiguo : nawpa
adorar : mutsakuy
venerar : mutsakuy
adornar : allitsay
arreglar : allitsay
agonizante : wanunaykaq
agravarse : antsay
agregar : yapay
repetir : yapay
agua : yaku
aguar : yakuchay
aguila : anka
aguja de arriero : antakasha
ahogamiento : shinqashyay
ají : utsu
ala : ripra
alacrán : atuq kuru
alegrarse : kushikuy
alga : kushuru
algodón : utku
aliso : ramrash
aliso : wayaw
alma : hupay
espíritu : hupay
almohada : hawya
alpaca : paqu
alumbrar : atsikyay
alumno : yachakuq
amanecer : waraq
amar : kuyay
amar : waylluy
amargar : puchquy
amargo : asqa
amarillo : qallwash
amasar : taqllay
amigo : yanasa
ampolla : pushllu
amuleto : illa
an

In [392]:
df = pd.DataFrame({
    'quechua': palabras_quechua,
    'espanol': palabras_espanol
})

# Mostrar el DataFrame
display(df)

,quechua,espanol
0,wanquyru,abeja
1,llaptsiy,ablandar
2,shulluy,abortar
3,waquy,abrazar
4,kichay,abrir
...,...,...
1138,muuka,zarigüeya
1139,shiraka,zarzaparrilla
1140,añas,zorrillo
1141,atuq,zorro


In [393]:
df.to_csv("diccionario.csv", index=False)